In [14]:
# NOTE: This is a temporary notebook used to develop the processing pipeline
# while waiting for Spark cluster issues to be resolved
# It's using pandas instead of spark

In [18]:
import jupyter_black

jupyter_black.load()

ModuleNotFoundError: No module named 'jupyter_black'

In [ ]:
import pandas as pd

df_raw = pd.read_json("sample_data.json", lines=True)
df_raw.head(10)

In [5]:
# Step 1: Filter out columns that will not be used

cols_to_keep = ["author", "body", "created_utc", "score", "subreddit", "subreddit_id"]

df_reddit = df_raw[cols_to_keep]
df_reddit.head(10)

,author,body,created_utc,score,subreddit,subreddit_id
0,Dethcola,A quarry,1506816000,3,sandiego,t5_2qq2q
1,PennyBotV2,[Salutations! I'm not sure what you said.](htt...,1506816000,3,RWBY,t5_2vhg0
2,Sir_Firebum,I got into baseball at about he same time Matt...,1506816000,2,baseball,t5_2qm7u
3,deanzynut,FUCKING TORY,1506816000,18,2007scape,t5_2wbww
4,OfullOstomacheO,I see a water dragon,1506816000,1,mildlyinteresting,t5_2ti4h
5,PlusOn3,Wait. The Michigan what? Where is this? Is thi...,1506816000,1,Cubers,t5_2r6a3
6,yeee_bot,ye fam,1506816000,2,teenagers,t5_2rjli
7,grrrrreat,143417804| &gt; United States Anonymous (ID: L...,1506816000,1,4chan4trump,t5_3hds7
8,psych4191,That is some chicken salad outta chicken shit ...,1506816000,2,CFB,t5_2qm9d
9,fishboy2000,Does he even know the rules?,1506816000,1,rugbyunion,t5_2qkbe


In [19]:
# Step 2.1: Count the biggest subreddits (by number of posts)

df_subred_count = pd.DataFrame(df_reddit.groupby("subreddit")["author"].count()).reset_index().rename(mapper={"author": "count"}, axis="columns")
df_subred_count.head(10)

,subreddit,count
0,1200isplenty,1
1,2007scape,22
2,2meirl4meirl,5
3,3DS,1
4,3Dprinting,3
5,3d6,1
6,3dshacks,4
7,40kLore,5
8,49ers,2
9,4PanelCringe,1


In [24]:
# Step 2.2: Get list with top 100 subreddit ids

subs_to_incl = 100

df_top_subs = df_subred_count.sort_values(by="count", ascending=False).iloc[0:subs_to_incl]
top_subs = df_top_subs["subreddit"].tolist()
print(top_subs)

['AskReddit', 'CFB', 'CrazyIdeas', 'news', 'ConciseIAmA', '4chan4trump', 'politics', 'RocketLeagueExchange', 'nba', 'The_Donald', 'edc_raffle', 'worldnews', 'funny', 'teenagers', 'leagueoflegends', 'Ice_Poseidon', 'DestinyTheGame', 'pics', 'hockey', 'nfl', 'gaming', 'videos', 'baseball', 'Jokes', 'Fireteams', 'rugbyunion', 'SquaredCircle', 'pcmasterrace', 'anime', 'relationships', 'LifeProTips', 'gifs', 'PoliticalHumor', 'fantasyfootball', 'MLS', 'Showerthoughts', 'DotA2', 'Rainbow6', 'MMA', 'mildlyinteresting', 'PUBATTLEGROUNDS', 'exmormon', 'europe', 'gonewild', 'movies', 'neoliberal', 'FireEmblemHeroes', 'aww', 'Overwatch', 'todayilearned', 'RocketLeague', 'GlobalOffensive', 'rupaulsdragrace', 'conspiracy', 'NBA2k', 'FIFA', '2007scape', 'btc', 'cars', 'soccer', 'hiphopheads', 'PS4', 'ChapoTrapHouse', 'FFBraveExvius', 'ffxiv', 'MaddenUltimateTeam', 'Guildwars2', 'CringeAnarchy', 'Bitcoin', 'magicTCG', 'food', 'sports', 'totalwar', 'destiny2', 'ColoradoRockies', 'CasualConversation', 

In [26]:
# Step 3.1: Filter data to only contain top subreddits

df_sub_filtered = df_reddit.loc[df_reddit["subreddit"].isin(top_subs)]
df_sub_filtered.head(10)

,author,body,created_utc,score,subreddit,subreddit_id
2,Sir_Firebum,I got into baseball at about he same time Matt...,1506816000,2,baseball,t5_2qm7u
3,deanzynut,FUCKING TORY,1506816000,18,2007scape,t5_2wbww
4,OfullOstomacheO,I see a water dragon,1506816000,1,mildlyinteresting,t5_2ti4h
6,yeee_bot,ye fam,1506816000,2,teenagers,t5_2rjli
7,grrrrreat,143417804| &gt; United States Anonymous (ID: L...,1506816000,1,4chan4trump,t5_3hds7
8,psych4191,That is some chicken salad outta chicken shit ...,1506816000,2,CFB,t5_2qm9d
9,fishboy2000,Does he even know the rules?,1506816000,1,rugbyunion,t5_2qkbe
10,raspberryseltzer,Tequila.,1506816000,2,CFB,t5_2qm9d
13,GonadusTwistus,you're really ignorant of history then. like ...,1506816000,2,The_Donald,t5_38unr
14,ithinkisaidtoomuch,lets see how deep this rabbit hole goes...,1506816000,1,CrazyIdeas,t5_2snxj


In [27]:
# Step 4.1: Find out who are active users

df_user_count = pd.DataFrame(df_sub_filtered.groupby("author")["body"].count()).reset_index().rename(mapper={"body": "count"}, axis="columns")
df_user_count.head(10)

,author,count
0,--Adam,1
1,-Dr-Mantis-Toboggan-,1
2,-TheDoodlebob,1
3,-damnittohell,2
4,-jb24,1
5,-redditardation-,1
6,-santman23-,1
7,-shadowstorm-,1
8,000TragicSolitude,2
9,0Catalyst,1


In [30]:
# Step 4.2: Find active and inactive users

comment_threshold = 1

df_top_users = df_user_count.loc[df_user_count["count"] > comment_threshold]
top_users = df_top_users["author"].tolist()
print(top_users)

['-damnittohell', '000TragicSolitude', '1000Airplanes', '1ain', '2crudedudes', '2minforgoaltending', '4kickz', '5225225', '69bit', '71stMB', 'AMaSTRIPPER_AMA', 'ARealKoala', 'ASmallCrane', 'AUfromthaBOOT', 'Aeolun', 'Aeschylus_', 'AfroClam', 'Amk5398', 'Archaeo', 'ArmoredSpearhead', 'AutoModerator', 'Aw_Frig', 'BASEDME7O', 'BBagot', 'BSnapZ', 'Bacxaber', 'Bamaborn97', 'BannerDay', 'Bartins', 'Beastbrook00', 'Beekeeper1987', 'BeerGogglesFTW', 'BenjaminWebb161', 'Bernie1400', 'BlueStripe100', 'Bobyyyyyyyghyh', 'BravoHotelTango', 'Brgisme', 'BucketsAMF', 'BuddyUpInATree', 'BullAlligator', 'CPGT', 'Caleb_26', 'Canux', 'CarolinaPunk', 'CaseofTrophies', 'Cheeba_Addict', 'Cheomesh', 'Chick-inn', 'ChickenwingChamp', 'Clever_Sardonic_Name', 'Clozer12', 'Cogeno', 'ColdWednesdayNight', 'Concise_AMA_Bot', 'CoolCatLovesAllKids', 'Cpt_Soban', 'CultOfMourning', 'D1amondDude', 'D4rkr4in', 'DaGoz', 'Dajja15', 'Daktush', 'Dalek1234', 'De1CawlidgeHawkey', 'DeemSleep', 'Deliwoot', 'DemonBurritoCat', 'Deni

In [32]:
# Step 4.3: Filter out inactive users with few comments

df_user_filtered = df_sub_filtered.loc[df_sub_filtered["author"].isin(top_users)]
df_user_filtered.head(10)

,author,body,created_utc,score,subreddit,subreddit_id
3,deanzynut,FUCKING TORY,1506816000,18,2007scape,t5_2wbww
4,OfullOstomacheO,I see a water dragon,1506816000,1,mildlyinteresting,t5_2ti4h
7,grrrrreat,143417804| &gt; United States Anonymous (ID: L...,1506816000,1,4chan4trump,t5_3hds7
8,psych4191,That is some chicken salad outta chicken shit ...,1506816000,2,CFB,t5_2qm9d
9,fishboy2000,Does he even know the rules?,1506816000,1,rugbyunion,t5_2qkbe
10,raspberryseltzer,Tequila.,1506816000,2,CFB,t5_2qm9d
14,ithinkisaidtoomuch,lets see how deep this rabbit hole goes...,1506816000,1,CrazyIdeas,t5_2snxj
15,Brgisme,"You are arguing to argue, I don't mind a story...",1506816000,2,NBA2k,t5_2s84e
22,halfhere,"They act like it’s some crazy idea, and the Fo...",1506816000,1,CFB,t5_2qm9d
23,AutoModerator,"***Welcome to /r/edc_raffle, please read the r...",1506816000,1,edc_raffle,t5_3jcb6


In [33]:
# Step 5: Create matrix with subreddit combinations

sub_combos = {(sub_1, sub_2): 0 for sub_1 in top_subs for sub_2 in top_subs}
print(sub_combos)

{('AskReddit', 'AskReddit'): 0, ('AskReddit', 'CFB'): 0, ('AskReddit', 'CrazyIdeas'): 0, ('AskReddit', 'news'): 0, ('AskReddit', 'ConciseIAmA'): 0, ('AskReddit', '4chan4trump'): 0, ('AskReddit', 'politics'): 0, ('AskReddit', 'RocketLeagueExchange'): 0, ('AskReddit', 'nba'): 0, ('AskReddit', 'The_Donald'): 0, ('AskReddit', 'edc_raffle'): 0, ('AskReddit', 'worldnews'): 0, ('AskReddit', 'funny'): 0, ('AskReddit', 'teenagers'): 0, ('AskReddit', 'leagueoflegends'): 0, ('AskReddit', 'Ice_Poseidon'): 0, ('AskReddit', 'DestinyTheGame'): 0, ('AskReddit', 'pics'): 0, ('AskReddit', 'hockey'): 0, ('AskReddit', 'nfl'): 0, ('AskReddit', 'gaming'): 0, ('AskReddit', 'videos'): 0, ('AskReddit', 'baseball'): 0, ('AskReddit', 'Jokes'): 0, ('AskReddit', 'Fireteams'): 0, ('AskReddit', 'rugbyunion'): 0, ('AskReddit', 'SquaredCircle'): 0, ('AskReddit', 'pcmasterrace'): 0, ('AskReddit', 'anime'): 0, ('AskReddit', 'relationships'): 0, ('AskReddit', 'LifeProTips'): 0, ('AskReddit', 'gifs'): 0, ('AskReddit', 'Po